
You can import the generated .ply file to a 3DGS viewer to view it.

If you use this web-based 3DGS viewer: https://projects.markkellogg.org/threejs/demo_gaussian_splats_3d.php 
The recommanded initial camera settings are:

- Camera up: 	        0,1,0
- Camera look-at:		0.05347, -0.15251, -0.64450
- Camera position:		0.00715, 0.17817, 0.71167


In [1]:
## Enviroment Setup
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # Set the visible CUDA, here we use the second GPU
WORKING_DIR = '/home/peizhi/Documents/gaussian-dejavu/'
os.chdir(WORKING_DIR) # change the working directory to the project's absolute path
print("Current Working Directory: ", os.getcwd())
sys.path.append(WORKING_DIR)
sys.path.append('./models')
sys.path.append('./networks')
sys.path.append('./utils')
sys.path.append('./utils/flame_lib/')
sys.path.append('./utils/diff-gaussian-rasterization')
sys.path.append('./utils/gaussian_renderer')
sys.path.append('./utils/scene')
sys.path.append('./utils/arguments')
sys.path.append('./utils/simple-knn')

import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm

import numpy as np
import cv2
import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchvision.transforms import Resize
# import math
# from torch.optim.lr_scheduler import StepLR

# # Dejavu
# from models.gaussian_dejavu_base import Framework
# from utils.loss import *
# from utils.scene.cameras import PerspectiveCamera, prepare_camera
# from utils.viewer_utils import OrbitCamera
# from utils.gaussian_renderer import render
# from utils.graphics_utils import create_diff_world_to_view_matrix, verts_clip_to_ndc
# from utils.loss_utils import *

# ## FLAME
# from utils.flame_lib import dict2obj
# from utils.flame_lib.FLAME import FLAME

import struct


# GUI
from ipywidgets import interactive, interact, Layout
from ipywidgets import FloatSlider
from ipywidgets import IntSlider
from ipywidgets import HBox, VBox, Label, Button


from dejavu import GaussianDejavu
from utils.personal_video_utils import *

def min_max_normalize(image):
    norm_image = (image - np.min(image)) / (np.max(image) - np.min(image))
    return norm_image

dejavu = GaussianDejavu(network_weights='./models/dejavu_network.pt', uv_map_size=120, num_expressions=20)

device = dejavu.device


## load head avatar

dejavu.load_head_avatar(save_path='./saved_avatars', avatar_name='peizhi-uv180')
#dejavu.load_head_avatar(save_path='./saved_avatars', avatar_name='imavatar-subject1-uv180')
#dejavu.load_head_avatar(save_path='./saved_avatars', avatar_name='imavatar-subject2-uv180')



Current Working Directory:  /home/peizhi/Documents/gaussian-dejavu
creating the FLAME Decoder


/home/peizhi/Documents/gaussian-dejavu/utils/flame_lib/FLAME.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('dynamic_lmk_faces_idx', torch.tensor(lmk_embeddings['dynamic_lmk_faces_idx'], dtype=torch.long))
/home/peizhi/Documents/gaussian-dejavu/utils/flame_lib/FLAME.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('dynamic_lmk_bary_coords', torch.tensor(lmk_embeddings['dynamic_lmk_bary_coords'], dtype=self.dtype))
/home/peizhi/miniconda3/envs/dejavu/lib/python3.10/site-packages/pytorch3d/io/obj_io.py:550: UserWarning: Mtl file does not exist: ./models/head_template2.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


Framework v3.1 initialized.
Number of Gaussians:  10694
model loaded from:  ./models/dejavu_network.pt
Gaussian DejaVu Framework Created.
Head avatar parameters loaded
Size of UV Gaussian map changed to 180x180
Number of Gaussians:  23873


In [ ]:
path = './saved_gaussians.ply'

# drive head avatar (exp = None for neutral expression)
rendered, gaussians = dejavu.drive_head_avatar(exp = None, pose = None, eye_pose = None, cam_pose = None, return_gaussians = True)

# save to .ply
gaussians[0].save_ply(path)

"""
Note: If you use this online viewer https://projects.markkellogg.org/threejs/demo_gaussian_splats_3d.php 
please use the following camera parameters:
Camera up: 	        0,1,0
Camera look-at:		0.05347, -0.15251, -0.64450
Camera position:	0.00715, 0.17817, 0.71167
"""
